## Model Training

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [12]:
df.drop(labels='id', axis=1, inplace=True)

In [13]:
x = df.drop(labels='price', axis=1)
y = df[['price']]

In [21]:
numCols = x.select_dtypes(exclude='object').columns
catCols = x.select_dtypes(include='object').columns
numCols, catCols

(Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object'),
 Index(['cut', 'color', 'clarity'], dtype='object'))

In [22]:
cutCat = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
colCat = ['D','E','F','G','H','I','J']
claCat = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [24]:
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)


cat_pipeline = Pipeline(
    steps=(
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[cutCat, colCat, claCat])),
        ('scaler', StandardScaler())
    )
)

preprocessor = ColumnTransformer(
    [
        ('num_pipeline', num_pipeline, numCols),
        ('col_pipeline', cat_pipeline, catCols)
    ]
)

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [26]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns=preprocessor.get_feature_names_out())

In [27]:
x_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,col_pipeline__cut,col_pipeline__color,col_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [28]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [29]:
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

In [30]:
regressor.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])

In [31]:
regressor.intercept_

array([3976.8787389])

In [32]:
import numpy as np 
def evaluate_model(true, predict):
    mse = mean_squared_error(true, predict)
    mae = mean_absolute_error(true, predict)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predict)
    return mse, mae, rmse, r2

In [33]:
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

trained_model = []
model_list = []
r2_list = []

for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    mse, mae, rmse, r2 = evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])

    model_list.append(list(models.keys())[i])

    print("Model training performance")
    print('RMSE', rmse)
    print('MAE',mae)
    print("R2 Score", r2)

    r2_list.append(r2)

    print('*'*35)
    print('\n')



LinearRegression
Model training performance
RMSE 1014.6296630375463
MAE 675.0758270067483
R2 Score 0.9362906819996049
***********************************


Lasso
Model training performance
RMSE 1014.6591302750638
MAE 676.2421173665509
R2 Score 0.9362869814082755
***********************************


Ridge
Model training performance
RMSE 1014.6343233534415
MAE 675.1077629781366
R2 Score 0.9362900967491631
***********************************


ElasticNet
Model training performance
RMSE 1533.3541245902313
MAE 1060.9432977143008
R2 Score 0.8544967219374031
***********************************


